# インタラクティブに実行する

以下の流れで、PHYSBO をインタラクティブに実行することができます。

1. PHYSBO から次に実行するパラメータを得ます。
2. PHYSBO の外部で評価値を得ます。
3. 評価値をPHYSBOに登録します。

例えば、以下の様な場合に適しています。

- 人手による実験を行い、その評価値をPHYSBOに与えたい。
- simulator の実行を別プロセスで行うなど、柔軟に実行制御を行いたい。

## 探索候補データの準備

In [1]:
import physbo

import os
import urllib2
import ssl
import numpy as np

ssl._create_default_https_context = ssl._create_unverified_context

def download():
    if not os.path.exists('data/s5-210.csv'):

        if not os.path.exists('data'):
            os.mkdir('data')
            
        print('Downloading...')
        response = urllib2.urlopen("http://www.tsudalab.org/files/s5-210.csv")
        with open('data/s5-210.csv', 'wb') as out_file:
            out_file.write(response.read())
        response.close()
        print('Done')
        
def load_data():
    download()
    A =  np.asarray(np.loadtxt('data/s5-210.csv',skiprows=1, usecols=(3,4,5,6), delimiter=',') )
    X = A[:,0:3]
    t  = -A[:,3]
    return X, t

X, t = load_data()
X = physbo.misc.centering(X)

## simulator の定義

In [2]:
class simulator:
    def __init__( self ):
        _, self.t = load_data()
    
    def __call__( self, action ):
        return self.t[action]

## 最適化の実行

In [3]:
# policy のセット 
policy = physbo.search.discrete.policy(test_X=X)

# シード値のセット 
policy.set_seed( 0 )

各探索ステップでは以下の処理を行っています。

1. `max_num_probes=1, simulator=None` として random_search または bayes_search を実行して actions を得る。
2. `t  = simulator(actions)` により評価値(の array) を得る。
3. `policy.write(actions, t)`により action に対する評価値を登録する。
4. `physbo.search.utility.show_search_results` により履歴を表示する。

In [ ]:
simulator = simulator()

''' 1st step (random sampling) '''
actions = policy.random_search(max_num_probes=1, num_search_each_probe=10, simulator=None)
t  = simulator(actions)
policy.write(actions, t)
physbo.search.utility.show_search_results(policy.history, 10)

''' 2nd step (random sampling) '''
actions = policy.random_search(max_num_probes=1, num_search_each_probe=10, simulator=None)
t = simulator(actions)
policy.write(actions, t)
physbo.search.utility.show_search_results(policy.history, 10)

''' 3rd step (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, num_search_each_probe=10, 
                                                      simulator=None, score='EI', interval=0,  num_rand_basis = 0)
t = simulator(actions)  
policy.write(actions, t) 
physbo.search.utility.show_search_results(policy.history, 10) 

''' 4-th step (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, num_search_each_probe=10, 
                                                      simulator=None, score='EI', interval=0,  num_rand_basis = 50)
t = simulator(actions) 
policy.write(actions, t)
physbo.search.utility.show_search_results(policy.history, 10)

## 中断と再開

以下の predictor, training, history を外部ファイルに保存することで、最適化プロセスを中断し、途中から再開することができます。

- predictor: 目的関数の予測モデル
- training: predictor の学習に用いるデータ (`physbo.variable` オブジェクト）
- history: 最適化実行の履歴 (`physbo.search.discrete.results.history` オブジェクト)

In [5]:
import pickle

with open('predictor.dump', 'wb') as f:
    pickle.dump(policy.predictor, f)
policy.training.save('training.npz')
policy.history.save('history.npz')

In [6]:
# policy を削除
del policy

# 保存した policy をロード 
policy = physbo.search.discrete.policy(test_X=X)
policy.load('history.npz', 'training.npz', 'predictor.dump')

''' 5-th probe (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, num_search_each_probe=10, simulator=None, score='EI', interval=0,  num_rand_basis = 0)
t = simulator(actions) 
policy.write(actions, t) 
physbo.search.utility.show_search_results(policy.history, 10) 

# predictor と training を個別に指定することも可
''' 6-th probe (bayesian optimization) '''
actions = policy.bayes_search(max_num_probes=1, num_search_each_probe=10, 
                                            predictor=policy.predictor, training=policy.training,
                                            simulator=None, score='EI', interval=0,  num_rand_basis = 0)
t = simulator(actions) 
policy.write(actions, t) 
physbo.search.utility.show_search_results(policy.history, 10) 

Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0 -th epoch marginal likelihood -27.81016746312926
50 -th epoch marginal likelihood -38.515180658606
100 -th epoch marginal likelihood -44.38603995143297
150 -th epoch marginal likelihood -47.687639627010014
200 -th epoch marginal likelihood -49.63382459992471
250 -th epoch marginal likelihood -50.830087948769304
300 -th epoch marginal likelihood -51.59732628884465
350 -th epoch marginal likelihood -52.11396264674491
400 -th epoch marginal likelihood -52.48246995632367
450 -th epoch marginal likelihood -52.76279457032902
500 -th epoch marginal likelihood -52.990202959467595
Done

0005-th multiple probe search (EI) 

current best f(x) = -0.959371 (best action = 6568) 
list of simulation results
f(x)=-0.959371 (action = 6568)
f(x)=-0.990530 (action = 5365)
f(x)=-0.996815 (action = 8990)
f(x)=-1.118381 (action = 2987)
f(x)=-1.009032 (action = 1643)
f(x)=-1.019266 (action = 11951)
f(x)=-1.033086 (a